# Agents

Agents use an LLM to determine which actions to take and in what order.
An action can either be using a tool and observing its output, or returning to the user.

When used correctly agents can be extremely powerful. The purpose of this notebook is to show you how to easily use agents through the simplest, highest level API. If you want more low level control over various components, check out the documentation for [custom agents](../custom_agents.ipynb).

## Concepts

In order to load agents, you should understand the following concepts:

- Tool: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains. The interface for a tool is currently a function that is expected to have a string as an input, with a string as an output.
- LLM: The language model powering the agent.
- Agent: The agent to use. This should be a string that references a support agent class. Because this notebook focuses on the simplest, highest level API, this only covers using the standard supported agents. If you want to implement a custom agent, see the documentation for custom agents (coming soon).

**For a list of supported agents and their specifications, see [here](../explanation/agents.md)**

## Tools
When constructing your own agent, you will need to provide it with a list of Tools that it can use. A Tool is defined as below.

```python
class Tool(NamedTuple):
    """Interface for tools."""

    name: str
    func: Callable[[str], str]
    description: Optional[str] = None
```

The two required components of a Tool are the name and then the tool itself. A tool description is optional, as it is needed for some agents but not all.

Besides defining tools yourself, you can also take advantage of predefined tools. For instructions on how to do that, please see later on in this notebook.

## Loading an agent


In [7]:
# Import things that are needed generically
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

In [8]:
# Load the tool configs that are needed.
from langchain import LLMMathChain, SerpAPIWrapper
llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    )
]

In [9]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [11]:
agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")



> Entering new AgentWithTools chain...
 I need to find out who Olivia Wilde's boyfriend is and then calculate his age raised to the 0.23 power.
Action: Search
Action Input: Olivia Wilde's boyfriend
Observation: Olivia Wilde started dating Harry Styles after ending her years-long engagement to Jason Sudeikis — see their relationship timeline.
Thought: I need to find out Harry Styles' age.
Action: Search
Action Input: Harry Styles age
Observation: 28 years
Thought: I need to calculate 28 raised to the 0.23 power.
Action: Calculator
Action Input: 28^0.23

> Entering new LLMMathChain chain...
28^0.23

```python
import math
print(math.pow(28, 0.23))
```

Answer: 2.1520202182226886

> Finished LLMMathChain chain.

Observation: Answer: 2.1520202182226886

Thought: I now know the final answer.
Final Answer: Harry Styles, Olivia Wilde's boyfriend, is 28 years old and his age raised to the 0.23 power is 2.1520202182226886.
> Finished AgentWithTools chain.


"Harry Styles, Olivia Wilde's boyfriend, is 28 years old and his age raised to the 0.23 power is 2.1520202182226886."

## Intermediate Steps
In order to get more visibility into what an agent is doing, we can also return intermediate steps. This comes in the form of an extra key in the return value, which is a list of (action, observation) tuples.

In [7]:
llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)

In [8]:
agent({"input":"How old is Olivia Wilde's boyfriend? What is that number raised to the 0.23 power?"})



> Entering new AgentWithTools chain...
 I need to find out how old Olivia Wilde's boyfriend is and then use a calculator to calculate the power.
Action: Search
Action Input: Olivia Wilde's boyfriend age
Observation: While Wilde, 37, and Styles, 27, have both kept a low profile when it comes to talking about their relationship, Wilde did address their ...
Thought: I now know the age of Olivia Wilde's boyfriend
Action: Calculator
Action Input: 27^0.23

> Entering new LLMMathChain chain...
27^0.23

```python
import math
print(math.pow(27, 0.23))
```

Answer: 2.1340945944237553

> Finished LLMMathChain chain.

Observation: Answer: 2.1340945944237553

Thought: I now know the final answer
Final Answer: 2.1340945944237553
> Finished AgentWithTools chain.


{'input': "How old is Olivia Wilde's boyfriend? What is that number raised to the 0.23 power?",
 'output': '2.1340945944237553',
 'intermediate_steps': [(AgentAction(tool='Search', tool_input="Olivia Wilde's boyfriend age", log=" I need to find out how old Olivia Wilde's boyfriend is and then use a calculator to calculate the power.\nAction: Search\nAction Input: Olivia Wilde's boyfriend age"),
   'While Wilde, 37, and Styles, 27, have both kept a low profile when it comes to talking about their relationship, Wilde did address their ...'),
  (AgentAction(tool='Calculator', tool_input='27^0.23', log=" I now know the age of Olivia Wilde's boyfriend\nAction: Calculator\nAction Input: 27^0.23"),
   'Answer: 2.1340945944237553\n')]}

## Predefined Tools
In addition to letting you define arbitrary tools, LangChain also provides some predefined tools. These can be either generic utility functions, chains, or other agents. You can easily load these tools by name (optionally specifying an LLM to use, if the tool requires it). For a list of all possible tools, please see [here](../../explanation/tools.md).

Let's recreate the above example loading tools by name. Because the LLMMath chain needs an LLM, we need to specify one.

In [4]:
from langchain.agents import load_tools
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [5]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [12]:
agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")



> Entering new AgentWithTools chain...
 I need to find out who Olivia Wilde's boyfriend is and then calculate his age raised to the 0.23 power.
Action: Search
Action Input: Olivia Wilde's boyfriend
Observation: Olivia Wilde started dating Harry Styles after ending her years-long engagement to Jason Sudeikis — see their relationship timeline.
Thought: I need to find out Harry Styles' age.
Action: Search
Action Input: Harry Styles age
Observation: 28 years
Thought: I need to calculate 28 raised to the 0.23 power.
Action: Calculator
Action Input: 28^0.23

> Entering new LLMMathChain chain...
28^0.23

```python
import math
print(math.pow(28, 0.23))
```

Answer: 2.1520202182226886

> Finished LLMMathChain chain.

Observation: Answer: 2.1520202182226886

Thought: I now know the final answer.
Final Answer: Harry Styles, Olivia Wilde's boyfriend, is 28 years old and his age raised to the 0.23 power is 2.1520202182226886.
> Finished AgentWithTools chain.


"Harry Styles, Olivia Wilde's boyfriend, is 28 years old and his age raised to the 0.23 power is 2.1520202182226886."